### 1.什么是卷积网络-CNN
    在使用全连接层时会出现的问题
        1)所有立体空间信息缺失
        2)立体空间信息缺失导致关键信息损失
        3)全像素入网导致计算量过大

通过滤波器(卷积核)以更小的代价提取图像上的特征--cnn

最大池化,平均池化

池化层必须放在卷积层后,卷积可以不需要池化不会影响运行

### 2.如何实现卷积神经网络

In [2]:
import keras #keras整框架导入
from keras.datasets import mnist #数据集
import matplotlib.pyplot as plt #matplotlib 用于打印输出图或图表
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D #神经网络层导入
from keras.models import Sequential #网络建设模式Sequential序列模型和Model函数模型
import numpy as np 


In [3]:
#加载官方mnist数据集
(x_train, y_train),(x_test, y_test) = mnist.load_data()
#（训练样本，训练样本的标签）（测试样本，测试样本的标签）

In [4]:
#归一化和数据reshape 与全连接层有区别
#第一部数据的预先处理之归一化 即将0-255之间的数据 压缩到0-1之间
x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

# 将原有的数据28*28 reshape到28,28,1,满足卷积层的数据入网要求(三维立体空间)
# 全连接层时reshape(60000, 784) 60000用-1代替,有多少算多少(不知道数据集有多大)

In [5]:
#标签转one-hot-encoding
y_train = keras.utils.to_categorical(y_train,num_classes=10)
y_test = keras.utils.to_categorical(y_test,num_classes=10)

In [6]:
#搭建网络结构 - 卷积神经网络
model = Sequential() #实例化
#像模型中添加卷积层为全连接层

model.add(Conv2D(
    filters = 10, #滤波器数量
    kernel_size = (3,3), #滤波器长度
    input_shape = (28, 28, 1), #输入图片尺寸
    strides = (1, 1), #滤波器步长
    activation = 'relu',
))

# 添加池化层
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# 添加打平层
model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.25)) #dropout是一个层，防止网络过度学习，比如学习了100个就保留一部分
model.add(Dense(units=10,activation='softmax'))

In [7]:
# 模型编译-设置模型的损失loss、优化器optimizier
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 启动训练,同时加入训练的样本,标签,训练数量和批次数量
model.fit(
    x = x_train,
    y = y_train,
    batch_size = 64, #60000/63=938
    epochs = 10,     #训练轮数
    
#     validation_split=0.1 #拿出10%训练样本作为验证样本，随着训练就开始验证
)

Epoch 1/10
938/938 [==============================] - 20s 18ms/step - loss: 0.2605 - accuracy: 0.9236
Epoch 2/10
938/938 [==============================] - 15s 16ms/step - loss: 0.0949 - accuracy: 0.9710
Epoch 3/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0666 - accuracy: 0.9798
Epoch 4/10
938/938 [==============================] - 13s 14ms/step - loss: 0.0510 - accuracy: 0.9847
Epoch 5/10
938/938 [==============================] - 14s 15ms/step - loss: 0.0414 - accuracy: 0.9872
Epoch 6/10
938/938 [==============================] - 15s 15ms/step - loss: 0.0351 - accuracy: 0.98870s - los
Epoch 7/10
938/938 [==============================] - 16s 17ms/step - loss: 0.0292 - accuracy: 0.9905
Epoch 8/10
938/938 [==============================] - 15s 16ms/step - loss: 0.0244 - accuracy: 0.9916
Epoch 9/10
938/938 [==============================] - 16s 17ms/step - loss: 0.0224 - accuracy: 0.9924
Epoch 10/10
938/938 [==============================] - 16s 17ms/step - los

In [8]:
model.save('../output/mnistTrain_cnn.h5')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 10)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1690)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               216448    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 217,838
Trainable params: 217,838
Non-trainable params: 0
__________________________________________________

In [10]:
loss, acc = model.evaluate(x_test, y_test)
print(loss, acc)

313/313 [==============================] - 2s 4ms/step - loss: 0.0430 - accuracy: 0.9870
0.04303539916872978 0.9869999885559082


In [11]:
import matplotlib.pyplot as plt
from keras.models import load_model
from PIL import Image
import numpy as np

In [12]:
### 加载DIY样本并进行预处理
img_open = Image.open('../data/3.jpg')

In [13]:
img_to_array = np.array(img_open).reshape(1,28,28,1)/255.0
print(img_to_array.shape)

(1, 28, 28, 1)


In [14]:
model = load_model('../output/mnistTrain_cnn.h5')
prediction = model.predict(img_to_array)
prediction

array([[2.2968724e-10, 1.0344683e-10, 3.6149670e-04, 9.9945754e-01,
        7.2623391e-10, 4.9003389e-05, 1.0259845e-08, 5.6749752e-05,
        7.5206597e-05, 1.4687100e-07]], dtype=float32)

In [15]:
Final_Pred = [result.argmax() for result in prediction]

In [16]:
print(Final_Pred)

[3]


In [17]:
counter = 0
for i in prediction[0]:
    percent = "%.2f%%"%(i*100)
    print(str(counter)+"概率",percent)
    counter += 1

0概率 0.00%
1概率 0.00%
2概率 0.04%
3概率 99.95%
4概率 0.00%
5概率 0.00%
6概率 0.00%
7概率 0.01%
8概率 0.01%
9概率 0.00%
